# Client Side


In [7]:
# importing library

import socket
import numpy as np
import cv2 as cv
import threading

In [8]:
# client program socket to connect to the server program
s = socket.socket()
s.bind(("", 9999))  # empty means local system
server_ip = input("enter server ip: ")
server_port = input("enter server port number: ")

s.connect((server_ip, int(server_port))) 
s.send(b"connected")  # sending string as data
camera = cv.VideoCapture(0) # starting the camera i.e. laptop webcam


# function for cleint to work as receiver
def receiver():
    framesLost = 0
    print("Entered")
    while True:
        framesLost += 1 # counting frame
        data = s.recv(100000000)  # receiving data with the size limit
        if(data == b'finished'): # to stop receiving and stop camera
            print("Finished")
            camera.release()
            s.close()
        else:  # converting the byte data into numpy array
            photo =  np.frombuffer(data, dtype=np.uint8)
            if len(photo) == 640*480*3: # changing the array shape and getting the video
                cv.imshow('From Server', photo.reshape(480, 640, 3))
                if cv.waitKey(1) == 13: # camera closing condition
                    s.send(b'finished')
                    camera.release()
                    cv.destroyAllWindows()
                    break
            else:
                print("Lost {} frames".format(framesLost) ) #counting the lost frames during video exchange

# function for client to send the data of the video             
def sender():
    while True: # reading the camera data resizing and sending it as byte 
        status, photo = camera.read()
        photo = cv.resize(photo, (640, 480))
        print(photo.shape)
        if status:
            s.send(np.ndarray.tobytes(photo))
        else: print("Could not get frame")
    camera.release()

# threads to run both the functions
threading.Thread(target=receiver).start()
threading.Thread(target=sender).start()

enter server ip: 192.168.39.122
enter server port number: 8080
Entered


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\JITHU HANSDA\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\JITHU HANSDA\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-8aeacdc74023>", line 42, in sender
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\JITHU HANSDA\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\JITHU HANSDA\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-8aeacdc74023>", line 18, in receiver
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


(480, 640, 3)
(480, 640, 3)
Lost 1 frames
Lost 2 frames
